# Sonification of Bleeding with Bank of Filters

# First Meeting (2019-06-11, Sasan and Thomas) @CITEC, TH proposed Filter-bank for feature generation
* The idea is to use a bank of different low-pass filters to create increasingly smooth signals
* these filtered signals serve as source for identifying key moments to anchor sound events
* which then create a multiscale data-driven complex grain structure of the raw instantaneous bleeding data.
* note that the limit of filtering with a cutoff-frequency towards 0 yields the integrated signal.

## Imports

In [1]:
from scipy import signal
import numpy as np
import scipy.interpolate
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt 
import copy

In [2]:
# %matplotlib inline

## Load Data and Create Filter-Bank Signals

In [3]:
df = pd.read_csv('log_refactored_correction_factor.csv', na_values=['no info', '.'], delimiter=',')
df_indexed = df.reset_index(drop=False)
index = df_indexed['index']
delta = df_indexed['Delta']
volume = df_indexed['Blood Accumulated']

delta_min = delta.min()
delta_max = delta.max()

volume_min = volume.min()
volume_max = volume.max()

print("dataset loaded:")
print(f"  delta:   min={delta_min:8}, max={delta_max:8.3}")
print(f"  volume:  min={volume_min:8}, max={volume_max:8}")

dataset loaded:
  delta:   min=     0.0, max=    8.59
  volume:  min=     0.0, max=  255.75


## Event-based Sonification of filtered data (min/max/threshold cut-throughs...)

In [4]:
import sc3nb as scn
import time
sc = scn.startup()

<IPython.core.display.Javascript object>

Starting sclang...
Done.
Registering UDP callback...
Done.
Booting server...
Done.
-> sc3nb started


In [5]:
#%sc FreqScope(400, 300)
#%sc s.makeGui
#%sc s.scope

In [6]:
import ipywidgets
import os
import threading
from IPython.display import clear_output
import copy

In [7]:
class Bloodplayer:
    
    def __init__(self, data, pulse_time=1, verbose=False):
        self.lock = threading.Lock()
        self.stopevent = threading.Event()
        self.callback_fn = None
        self.idx = 0
        self.data = data
        self.length = data.shape[0]
        self.verbose = verbose
        self.pulse_time = pulse_time
        
    #def __del__():
        # close plot window
        #pass
    
    def callback_fn_default(self, v):
        os.write(1, f"\r                       \r{v}".encode())
        
    def procfn(self):
        self.idx = 0
        while not self.stopevent.wait(0) and self.idx < self.length-1:
            v = self.data[self.idx]
            if self.verbose: 
                os.write(1, f"\r{self.idx}:{self.idx}                   ".encode())
            if callable(self.callback_fn):
                self.callback_fn(self)
            else:
                self.callback_fn_default(v)
            self.idx += 1
            time.sleep(self.pulse_time)
        print("done.")
    
    def set_callback(self, fn):
        self.callback_fn = fn
        
    def create_thread(self):
        threadname = "BloodPlayer-thread"
        # check first if it already exists
        if threadname in [t.name for t in threading.enumerate()]:
            print("create_thread: thread is already existing, stop first")
        else:
            self.stopevent.clear()
            self.producer = threading.Thread(name=threadname, target=self.procfn, args=[])
            self.producer.start()

    def stop_thread(self):
        self.stopevent.set()

In [8]:
bloodplayer = Bloodplayer(delta)

In [9]:
def voltau(tau=300):
    return np.hstack((np.zeros(tau), volume.values[tau:]-volume.values[:-tau]))/tau 
v5s = voltau(5)
v60s = voltau(60)
v300s = voltau(300)
print(v5s[20])

0.18868400000000002


In [10]:
# Plot Data 
%matplotlib

fig, ax = plt.subplots(1)  # create figure
mngr = plt.get_current_fig_manager(); 
mngr.window.setGeometry(840, 0, 600, 400)
plt.figure(1)
plt.clf()
# plt.plot(volume, 'g')
plt.plot(v5s, lw=1)
plt.plot(v60s, lw=1.2)
plt.plot(v300s, lw=1.5)
plt.plot(delta, lw=0.3)
plmarked, = ax.plot([], [], "r-", lw=1)


def update_plot(self, t, v1): 
    global fig, ax, plmarked, prev_idx
    
    plmarked.set_data([t,t], [-10, 10])
    #ax.draw_artist(ax.patch)
    #ax.draw_artist(plmarked)
    #ax.draw_artist(pldata)
    fig.canvas.update()

def onclick(event):
    global bloodplayer
    if event.dblclick:
        print(event.button, event.xdata)
        bloodplayer.idx = event.xdata


connection_id = fig.canvas.mpl_connect('button_press_event', onclick)


Using matplotlib backend: Qt5Agg


In [11]:
# GUI
def start(b):
    global bloodplayer
    bloodplayer.create_thread()
    print("start")
b1 = ipywidgets.Button(description='Start') 
b1.on_click(start)

def stop(b):
    global bloodplayer
    print("stop")
    bloodplayer.stop_thread()

b2 = ipywidgets.Button(description='Stop') 
b2.on_click(stop)
out = ipywidgets.Output()
ipywidgets.HBox([b1, b2, out])

start
stop
stop
done.
start
stop
done.


## Load Buffers

In [15]:
birds = sc.Buffer().load_file("samples/birds.wav")
rain = sc.Buffer().load_file("samples/rain.wav")
water_flow = sc.Buffer().load_file("samples/water-flow.wav")
crickets = sc.Buffer().load_file("samples/crickets.wav")

b = birds.bufnum
r = rain.bufnum
w = water_flow.bufnum
c = crickets.bufnum

## Sonification

In [17]:
# Custom code for sonifications

tau = [5, 60, 300]
v1 = 0
v2 = 0
v3 = 0

def son_waterdrop(self):
    global tau, v1, v2, v3
    
    # assign delta and volume
    delta_val = delta[self.idx]
    volume_val = volume[self.idx]

    # normalize delta and volume
    nd = scn.linlin(delta_val, delta_min, delta_max, 0, 1)       
    nv = scn.linlin(volume_val, volume_min, volume_max, 0, 1)
    
    if self.idx > tau[0]:
        v1 = (volume.values[self.idx] - volume.values[tau[0]])/tau[0]
        print(v1)
    
    os.write(1, f"\r{self.idx}, :callback{v1}       ".encode())
    
    update_plot(self,self.idx, v1)
    
bloodplayer.set_callback(son_waterdrop)

'''def voltau(tau=300):
return np.hstack((np.zeros(tau), volume.values[tau:]-volume.values[:-tau]))/tau 
v5s = voltau(5)
v60s = voltau(60)
v300s = voltau(300)
print(v5s[20])
'''


    


'def voltau(tau=300):\nreturn np.hstack((np.zeros(tau), volume.values[tau:]-volume.values[:-tau]))/tau \nv5s = voltau(5)\nv60s = voltau(60)\nv300s = voltau(300)\nprint(v5s[20])\n'

In [18]:
(volume.values[20] - volume.values[5])/5

0.22642

In [14]:
sc.msg("/s_new", "pb-1ch", 4567, 1, 1, "bufnum", b.bufnum, "rate", 1)

AttributeError: 'int' object has no attribute 'bufnum'

In [ ]:
sc.msg("/n_set", 4567, "amp", myfilter[self.idx])

In [14]:
a = pd.DataFrame([2,5,0,8,3,9,6,4,1,5,3])

In [15]:
w=a.values[:-3]
e=a.values[3:]
s=w-e
print(e)


[[8]
 [3]
 [9]
 [6]
 [4]
 [1]
 [5]
 [3]]


In [ ]:
-volume.values[:-tau]))/tau 